## Tuning starts here

In [46]:
import pandas as pd
import numpy as np

In [5]:
ppmi = pd.read_csv('./trans_processed_PPMI_data.csv')

/home/rz296/miniconda3/envs/partII/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246

In [6]:
ppmi.rename(columns={'Unnamed: 0':'Sentrix_position'}, inplace=True)
ppmi.set_index('Sentrix_position', inplace=True)
ppmi = ppmi.transpose()

## Run Classifier on original unreduced data without anything

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
label = encoder.fit_transform(ppmi['Category'])

In [8]:
tr = ppmi.drop(['Category'], axis=1)
X = tr.values
y = label
print(X.shape)
print(y.shape)

(436, 747668)
(436,)


In [9]:
#Stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
split.get_n_splits(X, y)

for train_index, test_index in split.split(X, y):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

TRAIN: 348 TEST: 88
(348, 747668) (348,) (88, 747668) (88,)


In [10]:
### Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
####所有的test都只能apply transform，不能用fit_transform!!!
X_test = scaler.transform(X_test)

## Tune parameters for Classifiers

### 1. Logistic Regression

In [8]:
# import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
#######Train model using X_train for regulariser and C value strength
###L1 first
C_options = [0.01, 0.1, 1, 1.5, 10, 100]

param_grid = [
    {
        'C': C_options,
    }
]

lr =  LogisticRegression(max_iter=500, penalty='l1', C=0.01, solver='saga')

grid = GridSearchCV(lr, param_grid=param_grid, scoring="accuracy", cv=6, n_jobs=6)
grid.fit(X_train, y_train)
y_pred = lr.predict
mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print('Best estimator: ', grid.best_params_)

In [ ]:
#######Train model using X_train for regulariser and C value strength
###L2 now
C_options = [0.01, 0.1, 1, 1.5, 10, 100]

param_grid = [
    {
        'C': C_options,
    }
]

lr =  LogisticRegression(max_iter=500, penalty='l2',solver='saga')

grid = GridSearchCV(lr, param_grid=param_grid, scoring="accuracy", n_jobs=5)
grid.fit(X_train, y_train)

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print('Best estimator: ', grid.best_params_)

In [ ]:
#With elasticnet
l1_ratio = [0.2, 0.5, 0.8]
param_grid = [
    {
        'l1_ratio': l1_ratio,
    }
]

lr =  LogisticRegression(max_iter=500, penalty='elasticnet',solver='saga')

grid = GridSearchCV(lr, param_grid=param_grid, scoring="accuracy", n_jobs=5)
grid.fit(X_train, y_train)

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print('Best estimator: ', grid.best_params_)

#### Notes:
- cross_val_score: used as an analysis tool to evaluate the results obtained by training strategy used before (i.e. the model may be applied entirely)
- cross_val_predict: apply cross-validation to training and get predictions and use it for analysis

### 2. SVM

In [9]:
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import Pipeline

In [123]:
####SGDClassifier with rbf kernel mapping

###Approx SVC, hence set penalty to be l2

# C_options = [0.01, 1, 100]
# kernels=['rbf', 'poly', 'linear', 'sigmoid']
# feature_map = Nystroem(gamma=1, random_state=1,n_components=300)
# svm = SGDClassifier(penalty='l2', loss='hinge', tol=0.1)
# svm_kernel_approx = Pipeline([
#     ("feature_map", feature_map),
#     ("svm", svm)
# ])

# param_grid = [
#     {
#         'feature_map__kernel': kernels,
#         'svm__alpha': C_options,
#     }
# ]

# svm_kernel_approx.fit(X_train, y_train)
# y_pred_svm_approx = svm_kernel_approx.predict(X_test) 

### Conclusion: SVM without regularisation has 
### worse performance than Logistic Regression

In [ ]:
###SVC as svm
###3 hypeparameters
kernels = ['rbf', 'poly', 'linear', 'sigmoid']
C_options=[0.01, 1, 1000]
gamma=[1e-4, 0.01, 1, 1.5]

param_grid = [
    {
        'C': C_options,
        'kernel': kernels,
        'gamma':gamma,
    }
]

grid = GridSearchCV(SVC(), param_grid=param_grid, scoring="accuracy", n_jobs=3)
grid.fit(X_train, y_train)

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print('Best estimator: ', grid.best_params_)

In [124]:
# print("Accuracy score of SVM:", accuracy_score(y_test, y_pred_svm))
# print("Accuracy score of SGDClassifier with kernel approx:", accuracy_score(y_test, y_pred_svm_approx))

Accuracy score of SVM: 0.7159090909090909
Accuracy score of SGDClassifier with kernel approx: 0.7045454545454546


### 3. XGBoost

In [8]:
import xgboost as xgb

In [ ]:
# Train XGBoost classifier
# DMatrix: a data structure that makes everything more efficient
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)
param = {'max_depth' : 3, 'eta' : 0.1, 'objective' : 'binary:logistic', 'seed' : 42, 'gpu_id': 0, 'tree_method': 'gpu_hist'}
num_round = 50
bst = xgb.train(param, dtrain, num_round, [(dtest, 'test'), (dtrain, 'train')])
# y_pred = bst.predict(dtest)
bst

## 1. Tune parameters for Dimensionality Reduction techniques + classifiers

### _1.1 PCA_

In [10]:
#######PCA on PPMI#########
from sklearn.decomposition import PCA

In [42]:
n_components = [50, 100, 150, 200, 250]
C_options = [0.01, 0.1, 1, 1.5, 10, 100]

#### 1.1.1 PCA+LR

In [48]:
### Tune n_components for PCA+Logistic Regression
###L1

pipe = Pipeline([
    ('pca', PCA()),
    ('clf', LogisticRegression(max_iter=500, penalty='l1'))
])

param_grid = [
    {
        'pca__n_components': n_components
        'clf__C': C_options
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

/home/rz296/miniconda3/envs/partII/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/rz296/miniconda3/envs/partII/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/sta

[0.65217391 0.66393375 0.53138716 0.49714286 0.52004141]
{'pca__n_components': 100}


/home/rz296/miniconda3/envs/partII/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
### Tune n_components for PCA+Logistic Regression
###L2

pipe = Pipeline([
    ('pca', PCA()),
    ('clf', LogisticRegression(max_iter=500, penalty='l2'))
])

param_grid = [
    {
        'pca__n_components': n_components
        'clf__C': C_options
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

#### 1.1.2 PCA+SVM

In [45]:
### Tune n_components for PCA+SVM

n_components = [50, 100, 150, 200, 250]
kernels = ['rbf', 'poly', 'linear', 'sigmoid']
C_options=[0.01, 1, 1000]
gamma=[1e-4, 0.01, 1, 1.5]

pipe = Pipeline([
    ('pca', PCA()),
    ('clf', SVC())
])

param_grid = [
    {
        'pca__n_components': n_components,
        'clf__C': C_options,
        'clf__kernel': kernels,
        'clf__gamma':gamma,
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy",n_jobs=3)
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print('Best estimator: ', grid.best_params_)


[0.61797101 0.61490683 0.62062112 0.63813665 0.63233954]
Best estimator:  {'pca__n_components': 200}



-----------
Conclusion so far:  
Applying PCA technique reduces the accuracy of model when only running on PPMI dataset  

-----------


### _1.2 UMAP_

In [58]:
from umap.umap_ import UMAP

### Tuning UMAP hyperparameters

#### 1.2.1 UMAP+LR

In [61]:
### Tune n_components for UMAP+Logistic Regression
###L1
n_components = [50, 100, 150, 200, 250]
C_options = [0.01, 0.1, 1, 1.5, 10, 100]

pipe = Pipeline([
    ('umap', UMAP()),
    ('clf', LogisticRegression(max_iter=500, penalty='l1'))
])

param_grid = [
    {
        'umap__n_components': n_components
        'clf__C': C_options
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

[0.68968944 0.68968944 0.68964803 0.68389234 0.6810766 ]
{'umap__n_components': 50}


In [ ]:
### Tune n_components for UMAP+Logistic Regression
###L2

pipe = Pipeline([
    ('umap', UMAP()),
    ('clf', LogisticRegression(max_iter=500, penalty='l2'))
])

param_grid = [
    {
        'umap__n_components': n_components
        'clf__C': C_options
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

#### 1.2.2 UMAP+SVM

In [62]:
### Tune n_components for UMAP+SVM

n_components = [50, 100, 150, 200, 250]
kernels = ['rbf', 'poly', 'linear', 'sigmoid']
C_options=[0.01, 1, 1000]
gamma=[1e-4, 0.01, 1, 1.5]

pipe = Pipeline([
    ('umap', UMAP()),
    ('clf', SVC())
])

param_grid = [
    {
        'umap__n_components': n_components,
        'clf__C': C_options,
        'clf__kernel': kernels,
        'clf__gamma':gamma,
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy",n_jobs=3)
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print('Best estimator: ', grid.best_params_)


[0.68674948 0.69540373 0.68679089 0.68968944 0.68964803]
Best estimator:  {'umap__n_components': 100}


### 1.3 ICA and Tune hyperparameter

In [63]:
from sklearn.decomposition import FastICA

In [64]:
### Tune n_components for ICA+Logistic Regression
###L1
n_components = [50, 100, 150, 200, 250]
C_options = [0.01, 0.1, 1, 1.5, 10, 100]

pipe = Pipeline([
    ('ica', FastICA()),
    ('clf', LogisticRegression(max_iter=500, penalty='l1'))
])

param_grid = [
    {
        'ica__n_components': n_components
        'clf__C': C_options
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

/home/rz296/miniconda3/envs/partII/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/rz296/miniconda3/envs/partII/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/sta

[0.67817805 0.62927536 0.64057971 0.61494824 0.65229814]


In [ ]:
### Tune n_components for ICA+Logistic Regression
###L2
n_components = [50, 100, 150, 200, 250]
C_options = [0.01, 0.1, 1, 1.5, 10, 100]

pipe = Pipeline([
    ('ica', FastICA()),
    ('clf', LogisticRegression(max_iter=500, penalty='l2'))
])

param_grid = [
    {
        'ica__n_components': n_components
        'clf__C': C_options
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

In [67]:
### Tune n_components for ICA+SVM

n_components = [50, 100, 150, 200, 250]
kernels = ['rbf', 'poly', 'linear', 'sigmoid']
C_options=[0.01, 1, 1000]
gamma=[1e-4, 0.01, 1, 1.5]

pipe = Pipeline([
    ('umap', UMAP()),
    ('clf', SVC())
])

param_grid = [
    {
        'umap__n_components': n_components,
        'clf__C': C_options,
        'clf__kernel': kernels,
        'clf__gamma':gamma,
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy",n_jobs=3)
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print('Best estimator: ', grid.best_params_)


[0.62662526 0.62658385 0.62650104 0.58339545 0.63817805]
Best estimator:  {'ica__n_components': 250}


## 2. Regularisation to FS for classification

In [1]:
###Use regularisation as Feature Selection technique to 
from sklearn.feature_selection import SelectFromModel

In [ ]:
####this returns the values that are positive after regularisation 
#Try different C value
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
sel_.fit(X_train, y_train)
#### sel_.get_support() returns a boolean matrix where True indicates the entries bigger than 0 and False otherwise
# selected_feat = X_train.columns[(sel_.get_support())]

In [ ]:
####Transform the original data to only the selected features based on regulariser
X_train_selected = sel_.transform(X_train)
X_test_selected = sel_.transform(X_test)

### 2.1 LR

In [ ]:
### Tune C value for regulariser for FS, then LR
###L1
C_options=[0.01, 1, 1000]

pipe = Pipeline([
    ('sel', SelectFromModel(LogisticRegression(C=1, penalty='l1'))),
    ('clf', LogisticRegression(max_iter=500, penalty='l1'))
])

param_grid = [
    {
        'sel__estimator__C': C_options #??
        'clf__C': C_options
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

In [ ]:
### Tune C value for regulariser for FS, then LR
###L2
C_options=[0.01, 1, 1000]

pipe = Pipeline([
    ('sel', SelectFromModel(LogisticRegression(C=1, penalty='l1'))),
    ('clf', LogisticRegression(max_iter=500, penalty='l2'))
])

param_grid = [
    {
        'sel__estimator__C': C_options, #??
        'clf__C': C_options
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

### 2.2 SVM

In [ ]:
### Tune C value for regulariser for FS, then SVM

C_options=[0.01, 1, 1000]
kernels = ['rbf', 'poly']

pipe = Pipeline([
    ('sel', SelectFromModel(LogisticRegression(C=1, penalty='l1'))),
    ('clf', SVC(max_iter=500))
])

param_grid = [
    {
        'sel__estimator__C': C_options, #??
        'clf__C': C_options,
        'clf__kernel': kernels
    },
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring="accuracy")
grid.fit(X_train, y_train)
# evaluation metric is accuray 

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print(mean_scores)
print(grid.best_params_)

 ## 3. VAE DR + CLF

In [19]:
import tensorflow as tf
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental_list_devices()

['/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:XLA_CPU:0',
 '/job:localhost/replica:0/task:0/device:XLA_GPU:0',
 '/job:localhost/replica:0/task:0/device:XLA_GPU:1',
 '/job:localhost/replica:0/task:0/device:XLA_GPU:2']

In [20]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
tf.test.is_gpu_available()

False

In [6]:
###Variational Autoencoder to get the latent layer
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Layer, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras import metrics, optimizers
from keras.callbacks import Callback
from keras.losses import mse, binary_crossentropy
import keras

import pydot
import graphviz
from keras.utils import plot_model
from keras_tqdm import TQDMNotebookCallback
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score


ppmi = pd.read_csv('./trans_processed_PPMI_data.csv')
ppmi.rename(columns={'Unnamed: 0':'Sentrix_position'}, inplace=True)
ppmi.set_index('Sentrix_position', inplace=True)
ppmi = ppmi.transpose()

encoder = LabelEncoder()
label = encoder.fit_transform(ppmi['Category'])

tr = ppmi.drop(['Category'], axis=1)
X = tr.values
y = label
print(X.shape)
print(y.shape)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
split.get_n_splits(X, y)

for train_index, test_index in split.split(X, y):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]



def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

/home/rz296/miniconda3/envs/partII/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246

(436, 747668)
(436,)
TRAIN: 348 TEST: 88
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 747668)       0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 400)          299067600   encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 100)          40100       dense_1[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 100)          40100       dense_1[0][0]                    
___________________________________________________

usage: ipykernel_launcher.py [-h] [-w WEIGHTS] [-m]
ipykernel_launcher.py: error: unrecognized arguments: -f /auto/homes/rz296/.local/share/jupyter/runtime/kernel-83d28ab7-a33d-4dbb-8605-396cf6fb50f4.json


SystemExit: 2

/home/rz296/miniconda3/envs/partII/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#Define custom variational layer
class CustomVariationalLayer(Layer):
    """
    Define a custom layer that learns and performs the training
    This function is borrowed from:
    https://github.com/fchollet/keras/blob/master/examples/variational_autoencoder.py
    """
    def __init__(self, **kwargs):
        # https://keras.io/layers/writing-your-own-keras-layers/
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x

In [ ]:
# Initialise variables and Set hyperparameters
original_dim = X.shape[1] #747668
latent_dim = 100

batch_size = 50 # controls the number of training samples to 
                # work through before the model's internal parameters are updated
epochs = 50



In [ ]:
# VAE model = encoder + decoder

############ENCODER##################
inputs = Input(shape=(original_dim, ), name='encoder_input')
#mean and log_var are the vectors of size `latent_dim`
z_mean_linear = Dense(latent_dim, kernel_initializer='glorot_uniform', name='z_mean')(inputs)
z_mean_batchnorm = BatchNormalization()(z_mean_linear)
z_mean_encoded = Activation('relu')(z_mean_batchnorm)

z_log_var_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(inputs)
z_log_var_batchnorm = BatchNormalization()(z_log_var_linear)
z_log_var_encoded = Activation('relu')(z_log_var_batchnorm)

# return the encoded and randomly sampled z vector
# Takes two keras layers as input to the custom sampling function layer with a `latent_dim` output
# note that "output_shape" isn't necessary with the TensorFlow backend

#### actually not very sure how and why use sampling??#####
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean_encoded, z_log_var_encoded])

# Model to compress input
encoder = Model(rnaseq_input, z_mean_encoded)


############DECODER##################
decoder_to_reconstruct = Dense(original_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
reconstructed_output = decoder_to_reconstruct(z)

########### instantiate VAE model##########
adam = optimizers.Adam(lr=learning_rate)
vae_layer = CustomVariationalLayer()([inputs, reconstructed_output])
vae = Model(inputs, vae_layer)
vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

vae.summary()


    
#Train the model
vae.fit(X_train, y_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, X_test))

score = vae.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 4. NN


### 4.1MLP

In [11]:
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

import talos
from talos.utils.gpu_utils import multi_gpu

import os

Using TensorFlow backend.


In [54]:
###Hyperparameters for MLP
p = {'lr': (0.8, 1.0, 3),
     'first_neuron':[256, 512],#,32, 64, 128
     'hidden_layer_neuron':[32], #64, 128, 256
     'batch_size': [10], #, 20, 30
     'epochs': [10], #, 20, 30
     'dropout': [0],#this means try every .1 value between 0 and .5
     'kernel_initializer': ['uniform'], #,'normal'
     'optimizer': ['Adam'], #'Nadam'
     'losses': ['binary_crossentropy'],
     'activation':['relu'], #, 'elu', 'tanh'
     'last_activation': ['sigmoid'] #, 'softmax'
    }

In [55]:
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="2";  
def mlp_model(x_train, y_train, x_val, y_val, params):
    # Build the model.
    # Anyhow give parameters first
    mlp = Sequential([
      Dense(params['first_neuron'], activation=params['activation'], input_shape=(747668,), kernel_initializer=params['kernel_initializer']),
      Dropout(params['dropout'], seed=42),
      Dense(params['hidden_layer_neuron'], activation=params['activation']),
      Dropout(params['dropout'], seed=42),
    #   Dense(8, activation='relu'),
    #   Dropout(Dropout(0.50, seed=42)),
      Dense(2, activation=params['last_activation']),
    ])

    # split a single job to multiple GPUs
#     mlp = multi_gpu(mlp)

    # Compile the model
    mlp.compile(
      optimizer=params['optimizer'],
      loss=params['losses'],
      metrics=['accuracy'],
    )

    # Train the data
    history = mlp.fit(
      x_train, # training data
      to_categorical(y_train), # training targets
      epochs=params['epochs'],
      batch_size=params['batch_size'],
    )

    return history, mlp

In [56]:
#Tune hyperparameters using talos
## To start fast, limit the permutation to 1/100 of the original permutation
scan_object = talos.Scan(x=X_train,
                         y=y_train, 
                         params=p,
                         model=mlp_model,
                         experiment_name='mlp')



  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/10
243/243 [==============================] - 22s 91ms/step - loss: 99.2364 - accuracy: 0.5720
Epoch 2/10
243/243 [==============================] - 22s 91ms/step - loss: 50.4819 - accuracy: 0.7737
Epoch 3/10
243/243 [==============================] - 22s 90ms/step - loss: 26.3427 - accuracy: 0.8148
Epoch 4/10
243/243 [==============================] - 22s 90ms/step - loss: 6.0931 - accuracy: 0.9300
Epoch 5/10
243/243 [==============================] - 22s 91ms/step - loss: 6.6742 - accuracy: 0.9465
Epoch 6/10
243/243 [==============================] - 22s 90ms/step - loss: 5.2511 - accuracy: 0.9486
Epoch 7/10
243/243 [==============================] - 22s 91ms/step - loss: 9.7458 - accuracy: 0.9342
Epoch 8/10
243/243 [==============================] - 22s 91ms/step - loss: 2.9741 - accuracy: 0.9527
Epoch 9/10
243/243 [==============================] - 22s 90ms/step - loss: 6.6655 - accuracy: 0.9465
Epoch 10/10
243/243 [==============================] - 22s 91ms/step - loss: 2.



 17%|█▋        | 1/6 [03:45<18:46, 225.38s/it]

Epoch 1/10
243/243 [==============================] - 22s 90ms/step - loss: 162.5748 - accuracy: 0.5597
Epoch 2/10
243/243 [==============================] - 22s 90ms/step - loss: 83.9551 - accuracy: 0.7922
Epoch 3/10
243/243 [==============================] - 22s 89ms/step - loss: 40.1386 - accuracy: 0.8683
Epoch 4/10
243/243 [==============================] - 22s 89ms/step - loss: 23.6784 - accuracy: 0.9321
Epoch 5/10
243/243 [==============================] - 22s 91ms/step - loss: 31.4243 - accuracy: 0.9403
Epoch 6/10
243/243 [==============================] - 22s 91ms/step - loss: 4.7200 - accuracy: 0.9588
Epoch 7/10
243/243 [==============================] - 22s 91ms/step - loss: 11.4621 - accuracy: 0.9506
Epoch 8/10
243/243 [==============================] - 22s 91ms/step - loss: 2.9521 - accuracy: 0.9712
Epoch 9/10
243/243 [==============================] - 22s 89ms/step - loss: 6.0054 - accuracy: 0.9630
Epoch 10/10
243/243 [==============================] - 22s 89ms/step - loss



 33%|███▎      | 2/6 [07:29<15:00, 225.00s/it]

...



 67%|██████▋   | 4/6 [18:41<09:43, 291.71s/it]

Epoch 1/10
243/243 [==============================] - 44s 182ms/step - loss: 171.5801 - accuracy: 0.5802
Epoch 2/10
243/243 [==============================] - 44s 181ms/step - loss: 45.4014 - accuracy: 0.7243
Epoch 3/10
243/243 [==============================] - 44s 181ms/step - loss: 9.8214 - accuracy: 0.7942
Epoch 4/10
243/243 [==============================] - 44s 180ms/step - loss: 8.6678 - accuracy: 0.8004
Epoch 5/10
243/243 [==============================] - 44s 181ms/step - loss: 2.4885 - accuracy: 0.8086
Epoch 6/10
243/243 [==============================] - 44s 181ms/step - loss: 2.0026 - accuracy: 0.7963
Epoch 7/10
243/243 [==============================] - 44s 181ms/step - loss: 1.3848 - accuracy: 0.8066
Epoch 8/10
243/243 [==============================] - 44s 183ms/step - loss: 0.3892 - accuracy: 0.8045
Epoch 9/10
243/243 [==============================] - 44s 179ms/step - loss: 0.3906 - accuracy: 0.8025
Epoch 10/10
243/243 [==============================] - 44s 181ms/step 



 83%|████████▎ | 5/6 [26:12<05:39, 339.51s/it]

Epoch 1/10
243/243 [==============================] - 43s 179ms/step - loss: 169.7645 - accuracy: 0.6111
Epoch 2/10
243/243 [==============================] - 44s 180ms/step - loss: 36.7431 - accuracy: 0.7922
Epoch 3/10
243/243 [==============================] - 44s 179ms/step - loss: 11.6468 - accuracy: 0.8683
Epoch 4/10
243/243 [==============================] - 44s 183ms/step - loss: 2.3361 - accuracy: 0.8807
Epoch 5/10
243/243 [==============================] - 44s 182ms/step - loss: 1.4793 - accuracy: 0.8992
Epoch 6/10
243/243 [==============================] - 44s 180ms/step - loss: 0.2897 - accuracy: 0.9198
Epoch 7/10
243/243 [==============================] - 44s 181ms/step - loss: 0.1192 - accuracy: 0.9239
Epoch 8/10
243/243 [==============================] - 45s 183ms/step - loss: 0.1588 - accuracy: 0.9239
Epoch 9/10
243/243 [==============================] - 44s 182ms/step - loss: 0.3978 - accuracy: 0.9177
Epoch 10/10
243/243 [==============================] - 45s 184ms/step



100%|██████████| 6/6 [34:00<00:00, 340.03s/it]


In [58]:
# accessing the results data frame
display(scan_object.data)

# accessing epoch entropy values for each round
display(scan_object.learning_entropy)

# access the summary details
scan_object.details

####Documentation for training: as the number of neuron increases, accuracy does not necessarily increase, however the time taken for training increases significantly

,round_epochs,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layer_neuron,kernel_initializer,last_activation,losses,lr,optimizer
0,10,2.722449,0.958848,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
1,10,2.506606,0.971193,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam
2,10,0.333380,0.936214,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.933333,Adam
3,10,0.121178,0.934156,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
4,10,0.389267,0.804527,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam
5,10,1.754559,0.911523,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.933333,Adam


,loss,accuracy
0,1.612864,2.292603
1,1.627731,2.292070
2,1.081373,2.292943
3,1.102660,2.294472
4,0.954518,2.298395
5,0.808678,2.296348


experiment_name                     mlp
random_method          uniform_mersenne
reduction_method                   None
reduction_interval                   50
reduction_window                     20
reduction_threshold                 0.2
reduction_metric                val_acc
complete_time            02/03/20/22:25
x_shape                   (348, 747668)
y_shape                          (348,)
dtype: object

In [59]:
sorted_so = scan_object.data.sort_values(by=["accuracy"], ascending=False)
sorted_so

,round_epochs,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layer_neuron,kernel_initializer,last_activation,losses,lr,optimizer
1,10,2.506606,0.971193,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam
0,10,2.722449,0.958848,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
2,10,0.333380,0.936214,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.933333,Adam
3,10,0.121178,0.934156,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
5,10,1.754559,0.911523,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.933333,Adam
4,10,0.389267,0.804527,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam


In [66]:
df1 = pd.DataFrame(sorted_so.iloc[:1])
# final_params_info = df1
df = pd.concat([df1, final_params_info]).sort_values(by=['accuracy'], ascending=False)
display(df)

final_params_info

,round_epochs,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layer_neuron,kernel_initializer,last_activation,losses,lr,optimizer
3,10,2.908766e-14,1.000000,relu,10,0,10,64,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
1,10,2.506606e+00,0.971193,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam


,round_epochs,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layer_neuron,kernel_initializer,last_activation,losses,lr,optimizer
3,10,2.908766e-14,1.0,relu,10,0,10,64,32,uniform,sigmoid,binary_crossentropy,0.8,Adam


In [31]:
#Clear GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()

In [11]:
# Evaluate on Test set
# evaluate() returns an array containing the test loss followed by any metrics we specified. 
mlp.evaluate(
  X_test,
  to_categorical(y_test)
)

88/88 [==============================] - 0s 3ms/step


[100.73245377974077, 0.6477272510528564]